In [6]:
# Imports

from ibpythonic import ibConnection, message
from ibapi.contract import Contract

from time import sleep, strftime
from datetime import datetime
import pickle
import os
from multiprocessing import Pool

import ib_data.workers

import pandas as pd
import numpy as np

from tinydb import TinyDB, Query

from tqdm import tqdm_notebook as tqdm

In [11]:
# Create contract object

def makeStkContract(contractTuple):
    newContract = Contract()
    newContract.symbol = contractTuple[0]
    newContract.secType = contractTuple[1]
    newContract.exchange = contractTuple[2]
    newContract.currency = contractTuple[3]
    return newContract

In [12]:
# Control variables declaration

bar_data_list = []
busy = False
active_contract_id = 0

In [13]:
# Historical data handler

def my_hist_data_handler(msg):
    global bar_data_list

    # If message does not belong to current contract, leave this function
    if int(msg.reqId) != active_contract_id:
        print('hist_data_handler: Msg does not belong to active contract: ' + str(msg))
        return
    
    # print(msg)
    bar_data_list.append(msg.bar)


In [14]:
# End of historical data handler

def my_hist_data_end_handler(msg):
    # print('hist_data_end_handler: ' + str(msg))
    global bar_data_list
    global busy
    global active_contract_id

    print('data retrieval complete.')
        
    # If message does not belong to current contract, leave this function
    if int(msg.reqId) != active_contract_id:
        print('hist_data_end_handler: Msg does not belong to active contract: ' + str(msg))
        return

    # Merge collected data into dataframe
    df_new = pd.DataFrame(columns=('date', 'open', 'high', 'low', 'close', 'volume'))
    for bar in bar_data_list:
        dashed_date = bar.date[:4] + '-' + bar.date[4:6] + '-' + bar.date[6:]
        row = {'date': dashed_date, 'open': bar.open, 'high': bar.high, 'low': bar.low, 'close': bar.close, 'volume': bar.volume}
        df_new = df_new.append(row, ignore_index=True)
    # df_new.loc[:, 'date'] = pd.to_datetime(df_new.date, format='%Y%m%d') #'%Y%m%d  %H:%M:%S'
    df_new = df_new.set_index('date')

    # Check if there is already existing data
    contracts_db = TinyDB('contracts_db.json')
    my_query = Query()
    result = contracts_db.search(my_query.Id == active_contract_id)
    contracts_db.close()
    current_contract = result[0]        
    contract_status = current_contract['Status']
    contract_symbol = current_contract['Symbol']
    contract_exchange = current_contract['Exchange']

    # Append to existing file or create new file
    filename = '/Users/martin/Google Drive/data/screener/' + contract_symbol + '_' + contract_exchange + '.csv'
    if contract_status.startswith('data_ends:'):
        df_old = pd.read_csv(filename, index_col='date')
        df_combined = df_new.combine_first(df_old)
        os.remove(filename)
        df_combined.to_csv(filename)
    else:
        if os.path.exists(filename):
            os.remove(filename)
        df_new.to_csv(filename)

    # write finished info to contracts database
        # Todo: use last day of data instead of today
    timestamp_now = datetime.now()
    string_now = timestamp_now.strftime('%Y-%m-%d') 
    contracts_db = TinyDB('contracts_db.json')
    my_query = Query()
    contracts_db.update({'Status': 'data_ends:'+string_now}, my_query.Id == active_contract_id)
    contracts_db.close()

    print(contract_symbol + '_' + contract_exchange + ' successful.')

    bar_data_list = []
    active_contract_id = 0
    busy = False

In [15]:
# Error handler

def my_error_handler(msg):
    # print('my_error_handler: ' + str(msg))
    
    global bar_data_list
    global busy
    global active_contract_id
    
    # If message does not belong to current contract, leave this function
    if int(msg.id) != active_contract_id:
        print('error_handler: Msg does not belong to active contract: ' + str(msg))
        return
        
    # write msg info to contracts database
    contracts_db = TinyDB('contracts_db.json')
    my_query = Query()
    status = 'Error:' + str(msg.errorCode) + '_' + str(msg.errorMsg)
    status = status.replace("'", "")
    contracts_db.update({'Status': status}, my_query.Id == active_contract_id)

    # get current contract info
    result = contracts_db.search(my_query.Id == active_contract_id)
    current_contract = result[0]        
    contract_symbol = current_contract['Symbol']
    contract_exchange = current_contract['Exchange']
    contracts_db.close()
    
    print(contract_symbol + '_' + contract_exchange + ' ' + status)
        
    bar_data_list = []
    active_contract_id = 0
    busy = False

In [40]:
# Data retrieval constants

endtime = strftime('%Y%m%d %H:%M:%S')
start_id = 5000
end_id = 5323

In [41]:
# Create connection object

con = ibConnection(port=7497)
con.register(my_hist_data_handler, message.historicalData)
con.register(my_hist_data_end_handler, message.historicalDataEnd)
con.register(my_error_handler, message.error)

True

In [ ]:
# Main function

con.connect()
    
for index in tqdm(range(start_id, end_id)):
        # Todo: Make the loop stoppable
        # Todo: Add timeout
    global busy
    global active_contract_id
    
    # Get contract data from contracts db
    contracts_db = TinyDB('contracts_db.json')
    my_query = Query()
    result = contracts_db.search(my_query.Id == index)
    contracts_db.close()
    current_contract = result[0]
    contract_status = current_contract['Status']       
    debug_string = str(index) + ': ' + current_contract['Symbol'] + '_' + current_contract['Exchange'] + ' requesting. '
    print(debug_string, end='')
    
    # Check if CSV file is already present
        # Todo: Skip contract on certain error status, eg. tws does not know this contract
        # todo: exit on certain error status, eg no connection
    if contract_status.startswith('data_ends:'):
        start_date = (contract_status.split(':'))[1]
        end_date = datetime.today().strftime('%Y-%m-%d')
        ndays = np.busday_count(start_date, end_date)
        if ndays <= 1:
            print('data is up to date. ' + str(ndays))
            continue
        ndays += 4
        req_string = str(ndays) + ' D'  
    else:
        req_string = "10 Y"
    
    if current_contract['Symbol'] == 'XGSD' and current_contract['Exchange'] == 'LSE':
        print('Skipping XGSD_LSE.')
        continue
        # Todo: Fix me
    
    # Create contract and request data
    contractTuple = (current_contract['Symbol'], 'STK', current_contract['Exchange'], current_contract['Currency'])
    stkContract = makeStkContract(contractTuple)
    active_contract_id = index
    busy = True
    con.reqHistoricalData(active_contract_id,stkContract,endtime,req_string,'1 day','MIDPOINT',1,1, keepUpToDate=False, chartOptions=[])
    
    # Wait for data download and storage
    while busy is True:
        sleep(0.2)
    sleep(0.2)
    print('-------------------------')
    
print('******** All done. ********')
con.disconnect()


In [10]:
 # Manual disconnection of connection object
    
con.disconnect()

True

# Enhancements

### Speedup
- download next contract as soon as data is in file

### Extra features for downloading
- check for splits/mergers
- Stream logger for status messages

### Symbol in multiple exchanges
- if symbol is multiple times in list
    - load data from primary exchange if ppossible
    - if primary is not available, load from exchange with highest average (price-)volume

### contracts database
- pull contracts from ib webpage
- pull contracts data from ib
- abillity to create own univestes (=contracts lists) based on contracts data

### Storage of price data
- store price data in database? sqlite, as handeled in one file? InfluxDB?
- cloud storage of data

### Misc
- 